In [4]:
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime
import io
import pandas as pd
import glob as gl

In [1]:
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

# URL of the the weather data download page.
url = 'https://dd.weather.gc.ca/climate/observations/hourly/csv/NS/'

# Station ID of the weather station of interest.
stn_id = '8202000'

# Destination directory to save the CSV files
save_dir = (r'C:\Users\daryl\OneDrive\Documents\GDAA3000\ProjectDischarge'
            r'\RdrsSample\LstmDatasets\NsWeatherData\Daily')

# Destination directory to save the concatenated CSV files.
dst_dir = (r'C:\Users\daryl\OneDrive\Documents\GDAA3000\ProjectDischarge'
            r'\RdrsSample\LstmDatasets\NsWeatherData\Concatenated')

# Send a GET request to the webpage.
response = requests.get(url)

# Parse the webpage with BeautifulSoup.
soup = BeautifulSoup(response.text, 'html.parser')

# Find all 'a' tags (which define hyperlinks) in the HTML
links = soup.find_all('a')

# Define the time period of interest.
start_date = datetime(2011, 1, 1)
end_date = datetime(2021, 12, 31)

# Loop over the links on the webpage.
for link in links:
    # Get the href attribute of the CSV links.
    if link.get('href').endswith('.csv'):
        # Extract the date from the filename. The date is the 5th element when splitting by '_'. 
        filename = link.get('href')
        date_str = filename.split('_')[4]  
        file_date = datetime.strptime(date_str, '%Y')

        # Check if the file is within the date range and has the correct prefix
        if stn_id in filename and start_date <= file_date <= end_date:
            # Construct the full URL to the CSV file.
            csv_url = url + filename

            # Send a GET request to the CSV URL.
            csv_response = requests.get(csv_url)

            # Create the name format for the saved CSV file. It will contain
            # the station ID and the year of the data.
            csv_name = filename.split('_')[3] + '_' + filename.split('_')[4] + '.csv'

            # Create the full path to the file
            file_path = os.path.join(save_dir, csv_name)

            # Write the contents of the response to a file
            with open(file_path, 'wb') as f:
                f.write(csv_response.content)

Concatenate the files

In [ ]:
# Get a list of all CSV files in the directory
csv_files = gl.glob(f'{save_dir} + /*{stn_id}*.csv')
dfs = []

for csv in csv_files:
    try:
        # Try to read the CSV file with 'utf-8' encoding
        with open(csv, 'rb') as f:
            content = f.read().decode('utf-8')
        df = pd.read_csv(io.StringIO(content))
    except UnicodeDecodeError:
        # If there's a UnicodeDecodeError, try a different encoding
        with open(csv, 'rb') as f:
            content = f.read().decode('ISO-8859-1')
        df = pd.read_csv(io.StringIO(content))
    dfs.append(df)
# Concatenate the dataframes.
df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all.drop(columns=['Longitude (x)', 'Latitude (y)', 'Station Name',
                              'Year', 'Month', 'Day', 'Data Quality', 'Max Temp Flag',
                               'Min Temp Flag', 'Mean Temp Flag', 'Heat Deg Days Flag',
                                'Cool Deg Days Flag', 'Total Rain Flag', 
                                'Total Snow Flag', 'Total Precip Flag', 
                                'Snow on Grnd Flag', 'Dir of Max Gust Flag',
                                   'Spd of Max Gust Flag'])

# Make Date/Time a datetime object
df_all['Date/Time'] = pd.to_datetime(df_all['Date/Time'])
df_all = df_all.rename(columns={'Date/Time': 'date'})

# Set the date as the index
df_all.set_index('date', inplace=True)

# Make the index regularly spaced (e.g., daily)
df_all = df_all.asfreq('D')

# Check if the index is regularly spaced. Returns 'True' if it is.
print(df_all.index.freq)

Save concatenated file.

In [ ]:
df_all.to_csv(os.path.join(dst_dir, stn_id + 'concatenated_daily.csv'))